# 01_data_merging.ipynb

**Purpose:**
Build a spatiotemporal modeling dataset to forecast human trafficking (HT) risk in NYC.

**This notebook:**
- Loads and cleans raw datasets (HT cases, HT crimes, events, passenger volumes).
- Creates a comprehensive daily-location grid for modeling.
- Engineers contextual features (crime density, event counts, holidays, etc).
- Merges everything for downstream LSTM and spatiotemporal models.

**Outputs:**
A ready-to-model CSV/Feather file with millions of rows, each representing a specific location on a specific day, enriched with context features.

> **Note:**  
> Outputs are cleared for readability. Heavy computations are not re-executed here.

In [ ]:
# Requirements: pandas, numpy, scikit-learn, holidays, tqdm
import sys
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.preprocessing import MinMaxScaler
import holidays
from tqdm import tqdm

print("Python version:", sys.version)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)


## 1. Data Loading
Load raw CSVs for HT cases, HT crimes, events, and passenger volumes.

In [ ]:
# Use relative paths as appropriate for your repo
ht_cases = pd.read_csv('../data/sample/TAHub_NYC_Metro_Cases.csv', low_memory=False)
ht_crimes = pd.read_csv('../data/sample/HT_RelatedCrimes_2021_2024.csv', low_memory=False)
passenger_volume = pd.read_csv('../data/sample/Passenger_volume_JFKinbound_2021_2024.csv', low_memory=False)
events = pd.read_csv('../data/sample/filtered_events_v2.csv', low_memory=False)


## 2. Parsing and Cleaning Dates/Times
Standardize and rename date/time columns for merging.

In [ ]:
# TAHub Cases
ht_cases['Incident Reporting Date'] = pd.to_datetime(ht_cases['Incident Reporting Date'])
ht_cases = ht_cases.rename(columns={'Incident Reporting Date': 'date'})

# HT Crimes
ht_crimes['CMPLNT_FR_DT'] = pd.to_datetime(ht_crimes['CMPLNT_FR_DT'])
ht_crimes = ht_crimes.rename(columns={'CMPLNT_FR_DT': 'date'})

# Passenger Volume
passenger_volume['Activity Period'] = pd.to_datetime(passenger_volume['Activity Period'])
passenger_volume = passenger_volume.rename(columns={'Activity Period': 'month'})

# Events
events['start_date'] = pd.to_datetime(events['start_date'])
events['end_date'] = pd.to_datetime(events['end_date'])
events = events.rename(columns={'start_date': 'date'})


## 3. Feature Engineering: Time Columns
Extract features like hour and night flags from crimes and events.

In [ ]:
# HT Crimes - parse hour
ht_crimes['crime_hour'] = pd.to_datetime(ht_crimes['CMPLNT_FR_TM'], format='%H:%M:%S', errors='coerce').dt.hour
# Events - parse start and end hour
events['event_start_hour'] = pd.to_datetime(events['start_time'], format='%H:%M:%S', errors='coerce').dt.hour
events['event_end_hour'] = pd.to_datetime(events['end_time'], format='%H:%M:%S', errors='coerce').dt.hour

# Night flags
ht_crimes['is_night_crime'] = ht_crimes['crime_hour'].apply(lambda x: 1 if x in range(0, 6) else 0)
events['is_night_event'] = events['event_start_hour'].apply(lambda x: 1 if x in range(0, 6) else 0)


## 4. Create Location IDs
Generate unique spatial IDs for each lat/lon pair.

In [ ]:
def make_location_id(lat, lon):
    return f"{round(lat, 5)}_{round(lon, 5)}"

ht_cases['location_id'] = ht_cases.apply(lambda r: make_location_id(r['Latitude'], r['Longitude']), axis=1)
ht_crimes['location_id'] = ht_crimes.apply(lambda r: make_location_id(r['Latitude'], r['Longitude']), axis=1)


## 5. Label Creation
Create the binary target: Will an HT case occur at this location tomorrow?

In [ ]:
# Create future label (future_ht_risk_1d)
future_flags = []
for loc, group in ht_cases.groupby('location_id'):
    for report_date in group['date']:
        future_flags.append((loc, report_date + pd.Timedelta(days=1)))
risk_df = pd.DataFrame(future_flags, columns=['location_id', 'date'])
risk_df['future_ht_risk_1d'] = 1

df_label = ht_cases[['date', 'Latitude', 'Longitude', 'location_id']].copy()
df_label = df_label.merge(risk_df, on=['location_id', 'date'], how='left')
df_label['future_ht_risk_1d'] = df_label['future_ht_risk_1d'].fillna(0).astype(int)


## 6. Build Spatiotemporal Grid
Cartesian product of all dates × all unique locations.

In [ ]:
from itertools import product
# Gather all unique lat/lon pairs
latlon_cases = ht_cases[['Latitude', 'Longitude']].dropna()
latlon_crimes = ht_crimes[['Latitude', 'Longitude']].dropna()
latlon_events = events[['Latitude', 'Longitude']].dropna() if 'Latitude' in events.columns else pd.DataFrame()

all_locations = pd.concat([latlon_cases, latlon_crimes, latlon_events], ignore_index=True).drop_duplicates()
all_locations['Latitude'] = all_locations['Latitude'].round(5)
all_locations['Longitude'] = all_locations['Longitude'].round(5)
all_locations['location_id'] = all_locations.apply(lambda r: make_location_id(r['Latitude'], r['Longitude']), axis=1)
all_locations = all_locations.drop_duplicates(subset=['location_id'])

# Date range
min_date = min(ht_cases['date'].min(), ht_crimes['date'].min(), events['date'].min())
max_date = max(ht_cases['date'].max(), ht_crimes['date'].max(), events['date'].max())
all_dates = pd.date_range(start=min_date, end=max_date)

# Cartesian product
date_loc_grid = pd.DataFrame(list(product(all_dates, all_locations['location_id'])), columns=['date', 'location_id'])
date_loc_grid = date_loc_grid.merge(all_locations, on='location_id', how='left')


## 7. Merge Contextual Features
- Event counts
- Past 7-day spatial crime count
- Night crime ratio
- Passenger volume
- Boroughs
- Temporal features (weekday, season, holidays)
- Supervised label

### 7.1. Merge Event Counts

In [ ]:
daily_event_counts = events.groupby('date').size().reset_index(name='daily_event_count')
date_loc_grid = date_loc_grid.merge(daily_event_counts, on='date', how='left')
date_loc_grid['daily_event_count'] = date_loc_grid['daily_event_count'].fillna(0).astype(int)


### 7.2. Add Past 7-Day Crime Count (within 1 mile)
Uses BallTree for efficient spatial queries.

**Note:** This section is computationally expensive for large datasets. For portfolio, you do not need to rerun it.

In [ ]:
EARTH_RADIUS_MILES = 3958.8
search_radius_miles = 1
search_radius_radians = search_radius_miles / EARTH_RADIUS_MILES

# Prepare coordinates in radians
ht_crimes = ht_crimes.dropna(subset=['Latitude', 'Longitude', 'date'])
ht_crimes['lat_rad'] = np.radians(ht_crimes['Latitude'])
ht_crimes['lon_rad'] = np.radians(ht_crimes['Longitude'])
crime_coords_rad = ht_crimes[['lat_rad', 'lon_rad']].to_numpy()
crime_dates = ht_crimes['date'].reset_index(drop=True)

date_loc_grid['lat_rad'] = np.radians(date_loc_grid['Latitude'])
date_loc_grid['lon_rad'] = np.radians(date_loc_grid['Longitude'])

crime_tree = BallTree(crime_coords_rad, metric='haversine')
crime_counts = []

# Example: Only run on a small sample for illustration (remove or adjust for full run)
# for i in tqdm(range(min(1000, len(date_loc_grid))), desc='Spatial 7d Crime Count'):
#    ...
# For full run, uncomment and beware of runtime
# for i in tqdm(range(len(date_loc_grid)), desc='Spatial 7d Crime Count'):
#    ...
# date_loc_grid['past_7d_crime_count'] = crime_counts

### 7.3. Night Crime Ratio (past 7 days, within 1 mile)

In [ ]:
# crime_hours = ht_crimes['crime_hour'].reset_index(drop=True)
# night_ratios = []
# for i in tqdm(range(len(date_loc_grid)), desc='Night Crime Ratio'):
#    ...
# date_loc_grid['night_crime_ratio'] = night_ratios

### 7.4. Merge Monthly Passenger Volume (scaled)

In [ ]:
if 'monthly_passenger_volume' not in passenger_volume.columns:
    passenger_volume = passenger_volume.rename(columns={'Revenue Passenger Volume': 'monthly_passenger_volume'})

date_loc_grid['month'] = date_loc_grid['date'].values.astype('datetime64[M]')
date_loc_grid = date_loc_grid.merge(
    passenger_volume[['month', 'monthly_passenger_volume']],
    on='month',
    how='left'
)
scaler = MinMaxScaler()
date_loc_grid['monthly_passenger_volume_scaled'] = scaler.fit_transform(
    date_loc_grid[['monthly_passenger_volume']].fillna(0)
)

### 7.5. Merge Boroughs
First try from HT Crimes, then supplement with address extraction from TAHub cases.

In [ ]:
borough_lookup = ht_crimes[['location_id', 'BORO_NM']].drop_duplicates()
date_loc_grid = date_loc_grid.merge(borough_lookup, on='location_id', how='left')
date_loc_grid = date_loc_grid.rename(columns={'BORO_NM': 'borough'})

def extract_borough_from_address(address):
    if pd.isna(address): return None
    address = address.upper()
    if "BRONX" in address: return "BRONX"
    elif "BROOKLYN" in address: return "BROOKLYN"
    elif "QUEENS" in address: return "QUEENS"
    elif "MANHATTAN" in address or "NEW YORK" in address: return "MANHATTAN"
    elif "STATEN ISLAND" in address: return "STATEN ISLAND"
    else: return None
ht_cases['borough_from_address'] = ht_cases['Address/Location'].apply(extract_borough_from_address)
borough_lookup_tahub = ht_cases[['location_id', 'borough_from_address']].dropna().drop_duplicates()
date_loc_grid = date_loc_grid.merge(borough_lookup_tahub, on='location_id', how='left')
date_loc_grid['borough'] = date_loc_grid['borough'].fillna(date_loc_grid['borough_from_address'])
date_loc_grid['borough'] = date_loc_grid['borough'].fillna("Unknown")
date_loc_grid = date_loc_grid.drop(columns=['borough_from_address'], errors='ignore')

### 7.6. Add Temporal Features

In [ ]:
date_loc_grid['day_of_week'] = date_loc_grid['date'].dt.dayofweek  # Monday=0
date_loc_grid['is_weekend'] = date_loc_grid['day_of_week'].isin([5, 6]).astype(int)
date_loc_grid['month_num'] = date_loc_grid['date'].dt.month

def get_season(month):
    if month in [12, 1, 2]: return 'Winter'
    elif month in [3, 4, 5]: return 'Spring'
    elif month in [6, 7, 8]: return 'Summer'
    else: return 'Fall'
date_loc_grid['season'] = date_loc_grid['month_num'].apply(get_season)

us_holidays = holidays.US(years=range(date_loc_grid['date'].dt.year.min(), date_loc_grid['date'].dt.year.max()+1))
date_loc_grid['is_holiday'] = date_loc_grid['date'].isin(us_holidays).astype(int)

### 7.7. Merge Supervised Label (future_ht_risk_1d)

In [ ]:
label_df = df_label[['date', 'location_id', 'future_ht_risk_1d']].drop_duplicates()
date_loc_grid = date_loc_grid.merge(label_df, on=['date', 'location_id'], how='left')
date_loc_grid['future_ht_risk_1d'] = date_loc_grid['future_ht_risk_1d'].fillna(0).astype(int)

### 7.8. Final Clean-up (drop duplicate/raw columns, check missing values)

In [ ]:
date_loc_grid = date_loc_grid.drop(
    columns=['monthly_passenger_volume', 'monthly_passenger_volume_x', 'monthly_passenger_volume_y',
            'month_num', 'lat_rad', 'lon_rad'], errors='ignore')
missing = date_loc_grid.isnull().sum()
print("Missing values (should all be zero):\n", missing[missing > 0])

## 8. Save Final Dataset
Write output for modeling. Use feather for fast reloads; csv for compatibility.

In [ ]:
date_loc_grid.to_feather("../data/final_modeling_dataset_v2.feather")
date_loc_grid.to_csv("../data/final_modeling_dataset_v2.csv", index=False)
print(f"✅ Saved final dataset: {date_loc_grid.shape} (feather & csv)")

## 9. Data Dictionary

| Column                         | Description                                               |
|--------------------------------|-----------------------------------------------------------|
| date                           | Day (YYYY-MM-DD)                                         |
| location_id                    | Rounded lat/lon identifier (e.g., '40.78343_-73.96625')  |
| Latitude, Longitude            | Spatial coordinates                                      |
| past_7d_crime_count            | Crimes within 1mi, past 7 days                           |
| night_crime_ratio              | Ratio of night crimes among past 7d crimes                |
| daily_event_count              | Number of city-permitted events that day                  |
| monthly_passenger_volume_scaled| Scaled inbound passenger volume (JFK)                     |
| borough                        | NYC borough                                              |
| day_of_week                    | Day of week (0=Mon)                                      |
| is_weekend                     | 1 if Saturday/Sunday, else 0                              |
| season                         | Season name                                              |
| is_holiday                     | 1 if US holiday, else 0                                  |
| future_ht_risk_1d              | 1 if HT case next day at location, else 0 (label)        |


**Next Steps:**  
- Exploratory data analysis (EDA) and visualizations were performed within this notebook and in the modeling notebook.
- See the modeling notebook (`02_modeling.ipynb`) for predictive modeling and further analysis.